# Define the feature_columns for DNNClassifier

#### Input
#### Output 
1. %store feature_columns


In [10]:
import tensorflow as tf
import peforth

### Define classifier 所需要的 feature_columns


In [12]:
%%f 讀進 Customer.txt 跟 BorrowerDEPT.txt 轉成 list 
char Customer.txt readTextFile trim :> split('\n') value Customers // ( -- list ) ['Annie', 'Astro', 'BENZ', ... ]
char BorrowerDEPT.txt readTextFile trim :> split('\n') value BorrowerDEPT // ( -- list ) ['13P2K0', '13P3K0', '1CK0K0', ... ]

In [13]:
Customer = tf.feature_column.categorical_column_with_vocabulary_list(
           'Customer', peforth.execute('Customers').pop())
BorrowerDEPT = tf.feature_column.categorical_column_with_vocabulary_list(
           'BorrowerDEPT', peforth.execute('BorrowerDEPT').pop())

# [ ] 2017-7 有 107 種 project name, 設定 200 個 bucket.
ProjectName = tf.feature_column.categorical_column_with_hash_bucket(
              'ProjectName', 200)

# PartNo 比 project name 多很多， 2018-7 有 561 種, 設定 700 個 bucket.
PartNo = tf.feature_column.categorical_column_with_hash_bucket('PartNo', 700)

# 沒看到 datetime type 不知道 date 該如何設定。比較法，與其用 string 不如用 integer --> integer 用不成， Barcode 太長，只能用 str。
Barcode = tf.feature_column.categorical_column_with_hash_bucket('Barcode', 2000) # 最老的 aging 有 1400 天，取 2000.

# QTY = tf.feature_column.numeric_column('QTY',dtype=tf.int32)     多說多錯
# Days = tf.feature_column.numeric_column('Days',dtype=tf.int32) ) 多說多錯
QTY = tf.feature_column.numeric_column('QTY')
Days = tf.feature_column.numeric_column('Days')


In [14]:
# DNNClassifier
# DNNClassifier 要求 "Items of feature_columns must be a _DenseColumn."

# 所謂 DenseColumn 就是把 categorical column 都包成 embedding_column or indicator_column. 所以前面的基礎上要再加工：

# Use indicator columns for low dimensional vocabularies
Customer2    = tf.feature_column.indicator_column(Customer)
BorrowerDEPT2        = tf.feature_column.indicator_column(BorrowerDEPT)

# Use embedding columns for high dimensional vocabularies
ProjectName2 = tf.feature_column.embedding_column(ProjectName,10)
PartNo2      = tf.feature_column.embedding_column(PartNo,10)
Barcode2     = tf.feature_column.embedding_column(Barcode,10)

# 打包順序不要緊
feature_columns = [Customer2, BorrowerDEPT2, ProjectName2, PartNo2, Barcode2, QTY, Days]

In [15]:
# 給將來 Training, Evaluating, Predicting 使用
%store feature_columns

Stored 'feature_columns' (list)


# 完成！